# Deploying the Hybrid Test Bench Emulator Service

In this notebook we will deploy a service that emulates the Hybrid Test Bench PT, and subsequently create a service to store data into the time series database.

To get our service running do the followning:
1. Make sure you're in the root of our repository ([hybrid-test-bench](./))
2. Run the following command: 

```bash
    python -m startup.start_services.py
   ``` 

This will start both InfluxDB and RabbitMQ services. This way we can have a consistent environment accross all machines.

Access, usernames and passwords are the following:

- **InfluxDB:**
  - Local site: [http://localhost:8086/](http://localhost:8086/)
  - User: bench
  - Password: bench

- **RabbitMQ:**
  - Local site: [http://localhost:15672/](http://localhost:15672/)
  - User: bench
  - Password: hybridtestbench

Now you should be able to access these two services, and we'll continue from here.

## Hybrid Test Bench (PT) Emulator Service

The interface of the emulator is very similar to the example for the Incubator ([IncubatorDTCourse\1-Incubator-Service\1-Incubator-Service.ipynb](..\1-Incubator-Service\1-Incubator-Service.ipynb)).

Our service will produce a RabbitMQ message every 3 seconds, and that message will contain data that matches exactly the data produced by the real Hybrid Test Bench - at least that is what we are trying to replicate.

The main difference is that, unlike the real Hybrid Test Bench, we will have to code the behavior that corresponds to applying a force on a specific beam. We will trigger this behavior using a RabbitMQ message that the emulator will listen for. This will lead to displacement, which we will show.


In [ ]:
#%%writefile logging.conf - we need to figure out if we need this?
""" [loggers]
keys=root,PTEmulatorService,IncubatorDataRecorderInflux

[handlers]
keys=consoleHandler,PTEmulatorServiceFileHandler,IncubatorDataRecorderInfluxFileHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler

[logger_PTEmulatorService]
level=DEBUG
handlers=PTEmulatorServiceFileHandler
qualname=PTEmulatorService
propagate=0

[logger_IncubatorDataRecorderInflux]
level=DEBUG
handlers=IncubatorDataRecorderInfluxFileHandler
qualname=IncubatorDataRecorderInflux
propagate=0

[handler_consoleHandler]
class=StreamHandler
formatter=simpleFormatter
args=(sys.stdout,)

[handler_PTEmulatorServiceFileHandler]
class=FileHandler
formatter=simpleFormatter
args=('PTEmulatorService.log', 'w')

[handler_IncubatorDataRecorderInfluxFileHandler]
class=FileHandler
formatter=simpleFormatter
args=('IncubatorDataRecorderInflux.log', 'w')

[formatter_simpleFormatter]
format=%(asctime)s.%(msecs)03d %(levelname)s %(name)s : %(message)s
datefmt=%Y-%m-%d %H:%M:%S """

In [ ]:
# Configure python path to load the hybrid test bench modules
import sys
import os

# Get the current working directory. Should be hybrid-test-bench
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'hybrid-test-bench', 'Current directory is not hybrid-test-bench'

# Get the parent directory. Should be the root of the repository
parent_dir = current_dir

# The root of the repo should contain the startup folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'startup')), 'startup folder not found in the repository root'

# The root of the repo should contain the installation folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'installation')), 'installation folder not found in the repository root'

# We don't need this for our case
""" incubator_dt_software_dir = os.path.join(parent_dir, 'incubator_dt', 'software')

assert os.path.exists(incubator_dt_software_dir), 'incubator_dt software directory not found'

# Add the parent directory to sys.path
sys.path.append(incubator_dt_software_dir) """